# Dropout and late sign-up handling

In [ ]:
import helper_functions as hf
from helper_functions import setup
from helper_functions.setup.openai_image_download import generate_all_images, save_resized_animal_images
import pandas as pd
hf.LOGGER.setLevel("INFO")


In [4]:
df = setup.sanitize_and_anonymize_data(overwrite=True, anonymize=True)
hf.DATA_NOW.reload()


[sports_week: WARNING] - Could not find the following people that payment was received from. Please check their names:
	['NaCu']
[sports_week: WARNING] - Found 4 people who managed to sign up multiple times. Keeping only  their latest entries.


In [5]:
dropout_list = []
# Calculate who's dropping out for what:
for _, row in hf.DATA_NOW.players.iterrows():
    if row["num_dropout_sports"] == 0:
        continue
    sports = []
    for sport in row["dropout_sports"]:
        if row[f"subteam_{sport}"] not in ["R", ""]:
            msg = f"{row['nickname']} ({row['Team']}) needs to be replaced for {sport}"
            sports.append(sport)
            print(msg)
            dropout_list.append( {
                "nickname": row["nickname"],
                "team": row["Team"],
                "sports": sports,
            })


Unwilling Starfish (Team A) needs to be replaced for beer_pong
Wealthy Armadillo (Team A) needs to be replaced for badminton
Neglected Harrier (Team B) needs to be replaced for running_sprints
Neglected Harrier (Team B) needs to be replaced for football
Neglected Harrier (Team B) needs to be replaced for capture_the_flag
Neglected Harrier (Team B) needs to be replaced for beer_pong
Unsteady Aardvark (Team B) needs to be replaced for tennis
Animated Yak (Team B) needs to be replaced for volleyball
Animated Yak (Team B) needs to be replaced for tennis
Animated Yak (Team B) needs to be replaced for ping_pong
Animated Yak (Team B) needs to be replaced for spikeball
Animated Yak (Team B) needs to be replaced for beer_pong
Affectionate Chihuahua (Team B) needs to be replaced for basketball
Ignorant Lemur (Team B) needs to be replaced for spikeball
Enormous Uakari (Team C) needs to be replaced for capture_the_flag
Enormous Uakari (Team C) needs to be replaced for beer_pong
Arctic Milkfish (Te

In [12]:
# Calculate whom of the new players would be available for what:
late = df[df["late_entry"]]
hf.DATA_NOW.reload()
print(f"Found {len(late)} late signups.")
no_team = late[~late["nickname"].isin(hf.DATA_NOW.players["nickname"].tolist())]
print(f"Of these, {len(no_team)} are not appointed to teams.")
for _, row in no_team.iterrows():
    avail_sports = [sport for sport in hf.DATA_NOW.sport_events if row[sport]]
    print(f"{row['nickname']} is available for {avail_sports}")


Found 6 late signups.
Of these, 1 are not appointed to teams.
Our Pheasant is available for ['chess']


In [9]:
# From the above, we can devise the following additions to teams:
teams_to_join = {
    "Velvety Seahorse": "B",
    "Tired Numbat": "B",
    "Bogus Wasp": "B",
    "Thorny Warthog": "A",
    "Well-to-do Moth": "A",
}


In [10]:
hf.DATA_NOW.reload()
for _, player in late.iterrows():
    name = player["nickname"]
    # Don't do anything if the player has already been added
    if any([team.contains_player(name) for team in hf.DATA_NOW.teams]):
        continue
    if name in teams_to_join:
        # team_to_join = teams[find_best_team_to_join(teams, player)]
        team_to_join = hf.DATA_NOW.get_team(teams_to_join[name])
        team_to_join.add_player(player, register_as_reserve=True)
        print(f"{name} joins {team_to_join.name}")
        generate_all_images([name])
# save_resized_animal_images(200)


Well-to-do Moth joins Team A
Image for Well-to-do Moth already exists, skipping.


In [13]:
# Now that players have been added to teams, we can start replacing dropouts in their subteams:
dropout_list

entry = dropout_list[0]
team = hf.DATA_NOW.get_team(entry["team"])
team_players = team.player_df
for sport in entry["sports"]:
    reserves = team_players[team_players[f"subteam_{sport}"] == "R"]
    if len(reserves) == 0:
        print(f"Currently no reserves available for {sport} in {team.name}")
        continue
    print(f"For {sport} in {team.name}, the following reserves to switch with {entry["nickname"]} are available: {reserves["nickname"].tolist()}")


For beer_pong in Team A, the following reserves to switch with Unwilling Starfish are available: ['Spicy Jaguar', 'Well-to-do Moth']


In [ ]:
hf.DATA_NOW.get_team("A").change_player_subteam("Well-to-do Moth", "beer_pong", player_to_replace_name="Unwilling Starfish")
hf.DATA_NOW.reload()


In [ ]:
hf.DATA_NOW.get_team("A").player_df[["nickname", "subteam_beer_pong"]]


,nickname,subteam_beer_pong
0,Stark Otter,6
1,Dishonest Fangtooth,
2,Striped Himalayan,3
3,Kooky Snake,6
4,Far-flung Sloth,1
...,...,...
58,Grimy Newfoundland,
59,Front Labradoodle,5
60,Unfortunate Salamander,
61,Sane Wallaby,1
